In [ ]:
!nvidia-smi

Wed Feb 17 09:42:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
pip install tensorflow-gpu


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
# import nlp
import os
import sys
import re

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import RNN, GRU, LSTM, Dense,Input, Embedding,Dropout, Activation, concatenate
from keras.layers import Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import initializers, regularizers, constraints, optimizers, layers


In [ ]:
 pip install nltk


In [ ]:
  pip install nlp

In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
embedding_file = 'glove.6B.300d.txt'

In [ ]:
train_data.describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
test_data.describe()

,id,comment_text
count,153164,153164
unique,153164,153164
top,a6273996063c8b8f,:*I'm thinking perhaps it should go more-or-le...
freq,1,1


In [ ]:
train_data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
test_data.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [ ]:
train_data.isnull().any()

id               False
comment_text     False
toxic            False
severe_toxic     False
obscene          False
threat           False
insult           False
identity_hate    False
dtype: bool

In [ ]:
test_data.isnull().any()

id              False
comment_text    False
dtype: bool

In [ ]:
classes = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y = train_data[classes].values

In [ ]:
train_sentences = train_data['comment_text'].fillna("fillna").str.lower()
test_sentences = test_data['comment_text'].fillna("fillna").str.lower()

In [ ]:
max_features = 100000
max_len = 150
embed_size =300

In [ ]:
tokenizer = Tokenizer(max_features)
tokenizer.fit_on_texts(list(train_sentences))

In [ ]:
tokenized_train_sentences = tokenizer.texts_to_sequences(train_sentences)  
tokenized_test_sentences = tokenizer.texts_to_sequences(test_sentences)

In [ ]:
train_sentences[1]

"d'aww! he matches this background colour i'm seemingly stuck with. thanks.  (talk) 21:51, january 11, 2016 (utc)"

In [ ]:
tokenized_train_sentences[1]

[96145,
 52,
 2635,
 13,
 555,
 3809,
 73,
 4556,
 2706,
 21,
 94,
 38,
 803,
 2679,
 992,
 589,
 8377,
 182]

In [ ]:
train_sentences[9]

'alignment on this subject and which are contrary to those of dulithgow'

In [ ]:
tokenized_train_sentences[9]

[10960, 15, 13, 242, 4, 53, 19, 1815, 2, 142, 3, 65704]

In [ ]:
train_padding = pad_sequences(tokenized_train_sentences, max_len)
test_padding = pad_sequences(tokenized_test_sentences, max_len)

In [ ]:
def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' '))for o in open(embedding_file,encoding="utf-8"))
# f = open(EMBEDDING_FILE,encoding="utf-8")
word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i>=max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
image_input = Input(shape=(max_len, ))
X = Embedding(max_features, embed_size, weights=[embedding_matrix])(image_input)
X = Bidirectional(GRU(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))(X)
# Dropout and R-Dropout sequence, inspired by Deep Learning with Python - Francois Chollet
avg_pl = GlobalAveragePooling1D()(X)
max_pl = GlobalMaxPooling1D()(X)
conc = concatenate([avg_pl, max_pl])
X = Dense(6, activation="sigmoid")(conc)
model = Model(inputs=image_input, outputs=X)

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
saved_model = "weights_base.best.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
early = EarlyStopping(monitor='val_acc', mode='max', patience=5)
callbacks_list=[checkpoint, early]

In [ ]:
batch_sz =32
epoch=2
model.fit(train_padding, y, batch_size = batch_sz, epochs=epoch, validation_split=0.1,callbacks=callbacks_list)

Epoch 1/2
4488/4488 [==============================] - 3581s 797ms/step - loss: 0.0694 - accuracy: 0.8884 - val_loss: 0.0443 - val_accuracy: 0.9039
Epoch 2/2
4488/4488 [==============================] - 3551s 791ms/step - loss: 0.0360 - accuracy: 0.9126 - val_loss: 0.0440 - val_accuracy: 0.9234


In [ ]:
test_values=model.predict([test_padding], batch_size=1024, verbose=1)

150/150 [==============================] - 163s 1s/step


In [ ]:
sample_submission = pd.read_csv('sample_submission.csv')
sample_submission[classes]=test_values
sample_submission.to_csv('submission.csv', index=False)